In [1]:
from urllib2 import Request, urlopen
from urlparse import urlparse, urlunparse
import requests, requests_cache
import pandas as pd
import json
from wordcloud import WordCloud
from matplotlib import pyplot as plt
plt.style.use('ggplot')
from bs4 import BeautifulSoup
import re
from os import listdir
from os import chdir
from os.path import isfile, join
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import string

In [2]:
r  = requests.get("http://abcnews.go.com/Politics")
data = r.text
soup = BeautifulSoup(data)

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file //anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
soup

<!DOCTYPE html>\n<!--[if IE 8 ]>    <html class="ie8" class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]--><!--[if IE 9 ]>    <html class="ie9" class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class=""> <!--<![endif]-->\n<head>\n<script>\n\tif (window.location.protocol == "https:" && window.parent.location.hostname.indexOf("outbrain") == -1) {\n\t\tvar _sslurl = window.location.href.replace("https://", "http://");\n\t\twindow.location.replace(_sslurl);\n\t\twindow.location.href = _sslurl;\n\t}\n</script>\n<title>Politics News | Breaking Political News, Video &amp; Analysis-ABC News - ABC News</title>\n<meta charset="unicode-escape"/>\n<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n<meta content="noarchive" name="robots"/>\n<meta content="index, follow" name="robots"/>\n<meta co

In [3]:
all_titles = soup.findAll('a', {'class': ['black-ln']})
titles = []
all_titles

[<a class="black-ln" data-analytics="cid=clicksource_77_null_headlines_hed" href="http://abcnews.go.com/Politics/trump-slash-state-department-budget-28-percent-shift/story?id=46151210">Trump wants to slash State Department budget by 28 percent</a>,
 <a class="black-ln" data-analytics="cid=clicksource_77_null_headlines_hed" href="http://abcnews.go.com/US/hawaii-judge-puts-trumps-travel-ban-hold/story?id=46157273">Judge puts Trump's revised travel ban on hold</a>,
 <a class="black-ln" data-analytics="cid=clicksource_77_null_headlines_hed" href="http://abcnews.go.com/Politics/read-full-white-house-budget-blueprint/story?id=46168992">Read the full White House budget blueprint</a>,
 <a class="black-ln" data-analytics="cid=clicksource_77_null_headlines_hed" href="http://abcnews.go.com/Politics/trump-budget-director-mick-mulvaney-money-build-wall/story?id=46166744">WH budget chief 'getting the money together to build the...</a>,
 <a class="black-ln" data-analytics="cid=clicksource_77_null_hea

In [4]:
for i in range(0, len(all_titles)):
    title = all_titles[i].getText()
    title = title.replace("\n", "")
    title = title.replace("\t", "")
    title = title.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22, 0x2026:0x20, 0x2014:0x20 }).encode('ascii', 'ignore')
    titles.append(title)
    
titles

['Spicer talks about what Trump meant in wiretapping tweets',
 "Conway doesn't have 'any evidence' of surveillance claims",
 "Democrat: Trump 'enthusiastic' about Medicare negotiating...",
 'Sen. Cotton: GOP risks House majority if health bill...',
 'Rep. Cummings: Dismissal of US attorney Bharara raises...',
 'GOP lawmaker defends racially charged tweet',
 "Dems, some GOP, condemn Bharara being 'fired'",
 'Intruder made it close to White House entrance',
 "US attorney says he was 'fired' after not resigning",
 '                    Latest Politics Video                ',
 '5 takeaways from the CBO health care report',
 'Trump plans to donate salary, despite pledge not to take one',
 "Congressional committee still waits for evidence of President Trump's Obama wiretapping claims",
 'DOJ asks for more time on inquiry into Trump wiretapping allegations',
 "HHS Sec. Tom Price: 'We disagree strenuously' with CBO report",
 'Trump orders review of executive branch',
 'CBO: 14 million more unin

In [4]:
def write_text(titles, file_name):
    """
    This function writes to a text file
    Input: titles and a filename
    Output: a new txt file in your directory
    """
    
    total_titles = titles
    title_file = open(file_name, "a")

    for i in range(0, len(total_titles)):
        temp = total_titles[i]
        title_file.write(temp + '\n')
    title_file.close()

In [30]:
write_text(x, "abctotal.txt")

In [10]:
titles[1]

'HHS secretary dodges on whether new health plan is...'

In [6]:
def trump_titles(title_names):
    """
    This function filters the article list with trump admin words
    Input: Unfiltered list of titles
    Output: Filtered list of titles
    """
    
    trumps_admin = ['whitehouse','Trump','conway','sessions','pence','president','tillerson','devos','flynn','kushner','carson','department','preibus','bannon','spicer','miller']

    total_titles = []

    for admin in trumps_admin:
        titles1 = [x for x in title_names if admin in x]
        total_titles.extend(titles1)

    return total_titles

In [18]:
def read_text(filename):
    """
    This function reads a txt file with filtered or unfiltered list of titles
    Input: A txt file
    Output: Filtered list of titles
    """
    
    title_names = []
    text_file = open(filename,"r")

    for line in text_file:
        line = line.strip()
        line = line.replace('"', '')
        line = line.replace("'", '')
        title_names.append(line)

    return trump_titles(title_names)

In [9]:
chdir('../sta141bfinal/abctitles/')

In [19]:
onlyfiles = [f for f in listdir('../abctitles/') if isfile(join('../abctitles/', f))] 

total_list = []
for i in range(0, len(onlyfiles)):
    trump_articles = read_text(onlyfiles[i])
    total_list = total_list + trump_articles

total_list

['Trump hasnt spoken to FBI dir. about wiretap claims: Spicer,',
 'Top Trumpcare takeaways, changes,',
 'Tracing where Trump gets some of his news ,',
 'Fact-checking Trump on Gitmo detainees,',
 'Top Trumpcare takeaways, changes,',
 'Trump has not spoken to FBI director about wiretapping claims, Spicer says,',
 'Spicer says Trump has not spoken to FBI director about wiretapping claims,',
 'Tracing where President Trump gets some of his news ,',
 'Fact-checking Trump on Guantanamo Bay detainees,',
 'What we know about Trumps unsubstantiated wiretapping allegations,',
 'President Trump surprises group on White House tour,',
 'Fist-pumping President Trump greets crowd of screaming kids,',
 'Trump very confident in his unsubstantiated wiretap claims, confidant says ,',
 'Trump, Netanyahu speak by phone, discuss Irans malevolent behavior,',
 'Exxon announces expansion program, Trump takes credit,',
 'Clapper denies Trump wiretap claim, calling it a distraction ,',
 'How Trumps new travel b

In [23]:
df = pd.DataFrame(total_list)
len(df)

115

In [24]:
x = df.drop_duplicates()

In [25]:
len(x)

73

In [28]:
x = x.iloc[:, 0].tolist()

In [29]:
len(x)

73